# Experiment: SharpeLoss (DMN style) Training & Equity Curve

- データ: `data/prices.csv`
- モデル: Decoder-Only TFT (Momentum Transformer)
- 目的: SharpeLoss最適化 + equity curve を2種類プロット


In [ ]:
# Setup
from __future__ import annotations

import sys
from pathlib import Path

root = Path.cwd()
if not (root / "mom_trans").exists():
    for p in root.parents:
        if (p / "mom_trans").exists():
            root = p
            break

if not (root / "mom_trans").exists():
    raise RuntimeError("Could not find repo root containing 'mom_trans'")

sys.path.insert(0, str(root))
# os.chdir(root)

import os
import random
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader

from mom_trans.data import build_datasets
from mom_trans.model import MomentumTransformer
from mom_trans.losses import SharpeLoss

# Matplotlib cache warning対策
os.makedirs('outputs/mpl', exist_ok=True)
os.environ['MPLCONFIGDIR'] = os.path.abspath('outputs/mpl')

import matplotlib.pyplot as plt

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# device
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('device:', device)


In [ ]:
# Data Load & Feature Engineering
seq_len = 252
train_ds, val_ds, test_ds, meta = build_datasets(
    data_path='data/prices.csv',
    seq_len=seq_len,
)

print('features:', meta.feature_cols)
print('train size:', len(train_ds))
print('val size:', len(val_ds))
print('test size:', len(test_ds))
print('split dates:', meta.split_dates)

batch_size = 64
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, drop_last=False)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, drop_last=False)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, drop_last=False)


In [ ]:
# Model & Training (SharpeLoss)
model = MomentumTransformer(
    num_features=len(meta.feature_cols),
    hidden_size=32,
    num_heads=4,
    dropout=0.1,
    output_mode="tanh",
).to(device)

loss_fn = SharpeLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

max_epochs = 100
patience = 5
best_sharpe = float('-inf')
patience_counter = 0
best_path = 'outputs/best_sharpe.pt'


def evaluate(loader):
    model.eval()
    losses = []
    with torch.no_grad():
        for inputs, targets, raw_next, time_idx, _ in loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            time_idx = time_idx.to(device)
            positions, _ = model(inputs)
            loss = loss_fn(positions, targets, time_idx)
            losses.append(loss.item())
    return float(np.mean(losses)) if losses else float('inf')

for epoch in range(1, max_epochs + 1):
    model.train()
    train_losses = []
    for inputs, targets, raw_next, time_idx, _ in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        time_idx = time_idx.to(device)

        optimizer.zero_grad()
        positions, _ = model(inputs)
        loss = loss_fn(positions, targets, time_idx)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_losses.append(loss.item())

    train_loss = float(np.mean(train_losses)) if train_losses else float('inf')
    val_loss = evaluate(val_loader)
    val_sharpe = -val_loss

    print(f"epoch {epoch}: train_loss={train_loss:.6f} val_loss={val_loss:.6f} val_sharpe={val_sharpe:.6f}")

    if val_sharpe > best_sharpe + 1e-4:
        best_sharpe = val_sharpe
        patience_counter = 0
        torch.save(model.state_dict(), best_path)
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print('Early stopping triggered')
            break

# load best
if os.path.exists(best_path):
    model.load_state_dict(torch.load(best_path, map_location=device))
    print('loaded best model')


In [ ]:
# Evaluation (Test only) - last time step per sequence
model.eval()
rows = []

idx_to_date = {v: k for k, v in meta.date_to_idx.items()}

with torch.no_grad():
    for inputs, targets, raw_next, time_idx, _ in test_loader:
        inputs = inputs.to(device)
        positions, _ = model(inputs)

        pos_last = positions[:, -1, 0].cpu().numpy()
        targ_last = targets[:, -1].numpy()
        raw_last = raw_next[:, -1].numpy()
        time_last = time_idx[:, -1].cpu().numpy().tolist()

        for i, t in enumerate(time_last):
            rows.append({
                'date': pd.Timestamp(idx_to_date[int(t)]),
                'vol_ret': pos_last[i] * targ_last[i],
                'raw_ret': pos_last[i] * raw_last[i],
                'eqw_ret': raw_last[i],
            })

if not rows:
    raise RuntimeError('No test rows collected')

results = pd.DataFrame(rows).sort_values('date')
results['equity_vol'] = (1.0 + results['vol_ret']).cumprod()
results['equity_raw'] = (1.0 + results['raw_ret']).cumprod()
results['equity_eqw'] = (1.0 + results['eqw_ret']).cumprod()
results.head()


In [ ]:
# Plot equity curves
plt.figure(figsize=(10, 5))
plt.plot(results['date'], results['equity_vol'], label='Strategy (Vol-adjusted)')
plt.plot(results['date'], results['equity_raw'], label='Strategy (Raw)')
plt.plot(results['date'], results['equity_eqw'], label='EQW (Raw)')
plt.title('Equity Curve (Test Only)')
plt.legend()
plt.tight_layout()

os.makedirs('outputs', exist_ok=True)
plot_path = 'outputs/equity_curve.png'
plt.savefig(plot_path, dpi=150)
plt.show()
print('saved:', plot_path)
